In [9]:
import pandas
import itertools as it
lookup_table = pandas.read_csv("./legacy/valid_values.csv", sep = ";")

print(lookup_table[0:3])

   NumChickens FieldArea ItemSurfaceArea
0           21      5625           19,24
1           21   6944,44           19,24
2           21   8402,78           19,24


In [10]:
lookup_table.reset_index()
for i, data in lookup_table.iterrows():
    if i > 0:
        break
    print(i)
    print(data)
    

0
NumChickens           21
FieldArea           5625
ItemSurfaceArea    19,24
Name: 0, dtype: object


In [11]:
from typing import Tuple, List
import math
import numpy as np
def compute_canonical_parameters(n_chickens: int, field_area: float, chicken_size: float) -> Tuple[float, float, float, float]:
    FA = (((0.006) * math.sqrt(field_area))**2)*math.pi
    

def compute_nd_nnd_coords(trial_left: List[float], trial_right: List[float]) -> Tuple[float, float]:
    #formula specified by previous thesis writers
    nnd_right = (0.577+0.467*trial_right[0])*(trial_right[1]/trial_right[0]) + (0.487+0.473*trial_right[0])*trial_right[2]
    nnd_left = (0.577+0.467*trial_left[0])*(trial_left[1]/trial_left[0]) + (0.487+0.473*trial_left[0])*trial_left[2]

    return np.log10(trial_right[0]/trial_left[0]), np.log10(nnd_right/nnd_left)

In [13]:
import random
data = {}
columns = ["Unnamed","NumLeft","FieldAreaLeft","ItemSurfaceAreaLeft","NumRight","FieldAreaRight","ItemSurfaceAreaRight","Ratio L/R","LogRatio","Log - Normalized","Difficulty Coefficient","Diff_coeff_filtering", "nd_LogRatio", "nnd_LogRatio"]

for c in columns:
    data[c] = []
i=0
for (ileft, dleft), (iright,dright) in it.combinations(lookup_table.sample(frac=1).iterrows(), 2):
    if dleft[0] == dright[0]:
        continue
    fAreaLeft = float(dleft[1].replace(",", "."))
    fAreaRight = float(dright[1].replace(",", "."))
    iAreaLeft = float(dleft[2].replace(",", "."))
    iAreaRight = float(dright[2].replace(",", "."))

    data["Unnamed"].append(i) 
    data["NumLeft"].append(dleft[0])
    data["FieldAreaLeft"].append(fAreaLeft)
    data["ItemSurfaceAreaLeft"].append(iAreaLeft)
    data["NumRight"].append(dright[0])
    data["FieldAreaRight"].append(fAreaRight)
    data["ItemSurfaceAreaRight"].append(iAreaRight)
    data["Ratio L/R"].append(-1)
    data["LogRatio"].append(-1)
    data["Log - Normalized"].append(-1)
    data["Difficulty Coefficient"].append(round(random.uniform(0, 1), 2))
    data["Diff_coeff_filtering"].append(round(random.uniform(0,1), 2))

    nd_logratio, nnd_logratio = compute_nd_nnd_coords([int(dleft[0]), fAreaLeft,iAreaLeft], [int(dright[0]), fAreaRight, iAreaRight])
    data["nd_LogRatio"].append(nd_logratio)
    data["nnd_LogRatio"].append(nnd_logratio)

In [15]:
new_table = pandas.DataFrame(data = data, columns=columns)
print(new_table)

       Unnamed  NumLeft  FieldAreaLeft  ItemSurfaceAreaLeft  NumRight  \
0            0       21       25069.44                93.11         9   
1            0       21       25069.44                93.11        29   
2            0       21       25069.44                93.11        13   
3            0       21       25069.44                93.11         5   
4            0       21       25069.44                93.11         9   
...        ...      ...            ...                  ...       ...   
27834        0       69       27777.78                19.24        81   
27835        0       69       27777.78                19.24         9   
27836        0       69       27777.78                19.24         9   
27837        0       81       36736.11                27.70         9   
27838        0       81       36736.11                27.70         9   

       FieldAreaRight  ItemSurfaceAreaRight  Ratio L/R  LogRatio  \
0            20069.44                222.38         -1 

In [16]:
import os
if os.path.exists("lookup_table.csv"):
    os.remove("lookup_table.csv")
new_table.to_csv("./lookup_table.csv")

In [17]:
lookup_table = pandas.read_csv("./lookup_table.csv")

print(lookup_table['nd_LogRatio'].min())
print(lookup_table['nd_LogRatio'].max())
print(lookup_table['nnd_LogRatio'].min())
print(lookup_table['nnd_LogRatio'].max())
#highest = lookup_table.iloc((lookup_table['nd_LogRatio']).abs().argsort()[:1])

-2.049218022670181
2.049218022670181
-0.8816194191065028
0.8477214271186774


In [23]:
okay = False
count = 0
count_two=0
for row in lookup_table.iterrows():
    row=row[1]
    if row["nd_LogRatio"] >0 and row["nnd_LogRatio"]<0:
        #print(row)
        okay = True
        #assert False == True
        count+=1
    if row["nd_LogRatio"] <0 and row["nnd_LogRatio"]>0:
        count_two +=1

print(okay)
print(count)
print(count_two)

True
5004
4030
